In [48]:
import cv2
import numpy as np

In [49]:
path = './test.jpg'
img = cv2.imread(path, 0)

In [50]:
def display(title, image):
    cv2.imshow(title, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    # cv2.waitKey(1)

### Image Negative

In [51]:
negative = 255 - img
display("Negative", negative)

### Log transform
Essentially brightens an image

In [69]:
# select a suitable constant c
max_pixel = float(np.max(img))
c = 255 / (np.log(1 + float(max_pixel)))  

# apply log transform
log_transformed = c * np.log(1 + img)  

# convert float to int
log_transformed = log_transformed.astype(np.uint8) 
 
display("Log transformed", log_transformed)


/var/folders/41/v7x6qrn91r5ccc86q_ps3c7c0000gn/T/ipykernel_53865/2377519716.py:6: RuntimeWarning: divide by zero encountered in log
  log_transformed = c * np.log(1 + img)
/var/folders/41/v7x6qrn91r5ccc86q_ps3c7c0000gn/T/ipykernel_53865/2377519716.py:9: RuntimeWarning: invalid value encountered in cast
  log_transformed = log_transformed.astype(np.uint8)
